In [1]:
import sys
sys.path.append("D:\\FundResearch")
import quant_utils.data_moudle as dm
import pandas as pd
from quant_utils.constant import DB_CONN_JJTG_DATA,DB_CONN_DATAYES, BARRA_SW21_FACTOR_NAME_DICT
from dateutil.parser import parse

不同分类基金规模变化

In [4]:
query_sql = """
WITH aa as ( SELECT a.*, ROUND(b.NET_ASSET_VALUE/100000000,2) AS NET_ASSET_VALUE, ROUND(b.end_share/100000000,2) AS END_SHARE, c.IS_MAIN AS IS_MAIN
FROM fund_type_own a 
JOIN fund_asset_own b ON (a.report_date = b.report_date) AND (a.TICKER_SYMBOL = b.TICKER_SYMBOL)
JOIN fund_info c ON (a.TICKER_SYMBOL = c.TICKER_SYMBOL)
ORDER BY report_date DESC, TICKER_SYMBOL
),
bb AS (SELECT LEVEL_1, LEVEL_2, report_date, sum(NET_ASSET_VALUE) AS sum_NET_ASSET_VALUE, sum(end_share) AS sum_END_SHARE
FROM aa 
GROUP BY LEVEL_1, LEVEL_2, report_date
ORDER BY LEVEL_1, LEVEL_2, report_date DESC),
cc AS (SELECT bb.*, LEAD(sum_NET_ASSET_VALUE, 1, NULL) OVER (PARTITION BY LEVEL_1, LEVEL_2) AS sum_NET_ASSET_VALUE_1, LEAD(sum_END_SHARE, 1, NULL) OVER (PARTITION BY LEVEL_1, LEVEL_2) AS sum_END_SHARE_1, LEAD(sum_NET_ASSET_VALUE, 4, NULL) OVER (PARTITION BY LEVEL_1, LEVEL_2) AS sum_NET_ASSET_VALUE_4, LEAD(sum_END_SHARE, 4, NULL) OVER (PARTITION BY LEVEL_1, LEVEL_2) AS sum_END_SHARE_4
FROM bb)
SELECT cc.*,
(sum_NET_ASSET_VALUE - sum_NET_ASSET_VALUE_1) AS asset_huanbi,
(sum_END_SHARE - sum_END_SHARE_1) AS share_huanbi,
(sum_NET_ASSET_VALUE - sum_NET_ASSET_VALUE_4) AS asset_tongbi,
(sum_END_SHARE - sum_END_SHARE_4) AS share_tongbi
FROM cc
WHERE report_date = 20230331
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [3]:
df

,LEVEL_1,LEVEL_2,REPORT_DATE,sum_NET_ASSET_VALUE,sum_END_SHARE,sum_NET_ASSET_VALUE_1,sum_END_SHARE_1,sum_NET_ASSET_VALUE_4,sum_END_SHARE_4,asset_huanbi,share_huanbi,asset_tongbi,share_tongbi
0,FOF,债券型FOF基金,2023-03-31,35.10,34.73,41.19,41.17,5.69,5.75,-6.09,-6.44,29.41,28.98
1,FOF,偏债混合型FOF基金,2023-03-31,910.58,868.62,1008.43,978.21,1303.11,1239.71,-97.85,-109.59,-392.53,-371.09
2,FOF,偏股混合型FOF基金,2023-03-31,491.89,503.85,487.52,512.31,477.94,487.43,4.37,-8.46,13.95,16.42
3,FOF,平衡混合型FOF基金,2023-03-31,156.02,149.46,150.16,147.87,130.58,125.31,5.86,1.59,25.44,24.15
4,FOF,目标日期型FOF基金,2023-03-31,218.20,179.47,195.25,164.07,159.12,122.52,22.95,15.40,59.08,56.95
5,FOF,股票型FOF基金,2023-03-31,15.39,16.43,12.90,15.01,11.04,12.21,2.49,1.42,4.35,4.22
6,QDII,ETF,2023-03-31,1919.45,2711.84,1704.65,2420.54,1242.11,1687.90,214.80,291.30,677.34,1023.94
7,QDII,ETF联接,2023-03-31,413.97,334.52,395.46,328.16,309.04,237.16,18.51,6.36,104.93,97.36
8,QDII,国际(QDII)REITs,2023-03-31,6.17,5.65,6.38,5.81,8.59,6.75,-0.21,-0.16,-2.42,-1.10
9,QDII,国际(QDII)偏股混合型基金,2023-03-31,423.52,256.49,434.82,272.27,433.22,278.17,-11.30,-15.78,-9.70,-21.68


季报2固收及固收加杠杆券种权益仓位变动

In [2]:
query_sql = """
WITH t ( REPORT_DATE, LEVEL1, LEVEL2, F_PRT_BONDTONAV, F_PRT_COVERTBONDTONAV, F_PRT_INTERESETBONDTONAV, F_PRT_CREDITBONDTONAV, EQUITY_IN_NA, HK_in_na ) AS (
	SELECT
	CASE
			grouping ( a.REPORT_DATE ) 
			WHEN 1 THEN
			'总计' ELSE a.REPORT_DATE 
		END,
	CASE
			grouping ( b.LEVEL_1 ) 
			WHEN 1 THEN
			'总计' ELSE b.LEVEL_1 
		END,
	CASE
			grouping ( b.LEVEL_2 ) 
			WHEN 1 THEN
			'--' ELSE b.LEVEL_2 
		END,
		avg( a.F_PRT_BONDTONAV ),
		avg( a.F_PRT_COVERTBONDTONAV ),
		avg( `a`.`F_PRT_GOVBONDTONAV` + `a`.`F_PRT_LOCALGOVBONDTONAV` + `a`.`F_PRT_CTRBANKBILLTONAV` + `a`.`F_PRT_POLIFINANBDTONAV` ),
		avg( `a`.`F_PRT_BONDTONAV` - `a`.`F_PRT_COVERTBONDTONAV` - `a`.`F_PRT_GOVBONDTONAV` - `a`.`F_PRT_LOCALGOVBONDTONAV` - `a`.`F_PRT_CTRBANKBILLTONAV` - a.F_PRT_POLIFINANBDTONAV ),
		avg( a.EQUITY_MARKET_VALUE / a.NET_ASSET * 100 ),
		avg( a.F_PRT_HKSTOCKVALUETONAV ) 
	FROM
		fund_asset_own a
		JOIN fund_type_own b ON b.REPORT_DATE = a.REPORT_DATE 
		AND a.TICKER_SYMBOL = b.TICKER_SYMBOL 
	WHERE
		b.LEVEL_1 IN ( '固收', '固收+' ) 
	GROUP BY
		a.REPORT_DATE,
		b.LEVEL_1,
		b.LEVEL_2 WITH ROLLUP 
	HAVING
		( grouping ( a.REPORT_DATE ) <> 1 ) 
	) SELECT
	REPORT_DATE as '报告期',
	level1 as '知己内部分类:一级',
	level2 as '知己内部分类:二级',
	round(F_PRT_BONDTONAV,2) + round(EQUITY_IN_NA,2) as '杠杆',
	round(F_PRT_COVERTBONDTONAV,2) as '可转债占净值',
	round(F_PRT_INTERESETBONDTONAV,2) as '利率债占净值',
	round(F_PRT_CREDITBONDTONAV,2) as '信用债占净值',
	round(EQUITY_IN_NA,2) as '权益占净值',
    round(EQUITY_IN_NA + 0.5*F_PRT_COVERTBONDTONAV,2) as '风险暴露'
FROM
	t 
WHERE
	level2 IS NOT NULL 
	and level1 != '总计' and level2 != '--'
	and report_date >= '20191231'
ORDER BY
	level1,
	level2,
	REPORT_DATE;
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df_leverage = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='杠杆')
df_insteret = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='利率债占净值')
df_credit = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='信用债占净值')
df_stock = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='权益占净值')
df_risk_exposure = df.pivot_table(columns='知己内部分类:二级', index='报告期', values='风险暴露')

In [3]:
df_risk_exposure

知己内部分类:二级,(00-05),(05-15),(15-25),(25-35),(35+),中长债,可转债,短债
报告期,,,,,,,,
2020-03-31,3.47,11.90,21.81,32.12,39.67,0.02,51.17,0.00
2020-06-30,2.84,12.07,21.26,30.17,41.21,0.02,53.61,0.00
2020-09-30,3.38,11.70,21.60,28.92,39.30,0.01,53.07,0.00
2020-12-31,3.10,12.42,21.53,29.47,37.97,0.01,53.42,0.01
2021-03-31,3.14,10.50,19.56,27.39,35.66,0.03,52.45,0.01
2021-06-30,3.13,11.25,19.70,28.38,41.02,0.02,50.42,0.01
2021-09-30,3.22,10.68,19.85,28.86,37.26,0.02,49.51,0.00
2021-12-31,3.20,12.18,20.77,30.89,40.70,0.02,51.03,0.01
2022-03-31,2.98,10.02,19.15,29.81,42.98,0.02,54.63,0.01


权益基金A股和港股仓位

In [5]:
query_sql = """
WITH aa AS (SELECT 
a.report_date AS REPORT_DATE,
a.TICKER_SYMBOL AS TICKER_SYMBOL,
a.SEC_SHORT_NAME AS SEC_SHORT_NAME,
a.LEVEL_1 AS LEVEL_1,
a.LEVEL_2 AS LEVEL_2,
a.LEVEL_3 AS LEVEL_3,
b.EQUITY_RATIO_IN_NA AS EQUITY_RATIO_IN_NA,
b.F_PRT_HKSTOCKVALUETONAV AS F_PRT_HKSTOCKVALUETONAV,
(b.EQUITY_RATIO_IN_NA - b.F_PRT_HKSTOCKVALUETONAV) AS F_PRT_CNSTOCKVALUETONAV,
c.IS_MAIN AS IS_MAIN 
FROM `fund_type_own` a
JOIN fund_asset_own b ON (a.REPORT_DATE = b.REPORT_DATE) AND (a.TICKER_SYMBOL = b.TICKER_SYMBOL)
JOIN fund_info c ON (a.TICKER_SYMBOL = c.TICKER_SYMBOL) AND (c.IS_MAIN = 1)
WHERE a.LEVEL_1 = "主动权益"
ORDER BY REPORT_DATE DESC)
SELECT report_date, avg(EQUITY_RATIO_IN_NA), avg(F_PRT_HKSTOCKVALUETONAV), avg(F_PRT_CNSTOCKVALUETONAV)
FROM aa 
GROUP BY report_date 
ORDER BY report_date 

"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [6]:
df

,REPORT_DATE,avg(EQUITY_RATIO_IN_NA),avg(F_PRT_HKSTOCKVALUETONAV),avg(F_PRT_CNSTOCKVALUETONAV)
0,2007-12-31,78.769502,0.000000,78.769502
1,2008-03-31,73.800640,0.000000,73.800640
2,2008-06-30,70.573955,0.000000,70.573955
3,2008-09-30,65.489564,0.000000,65.489564
4,2008-12-31,66.972099,0.000000,66.972099
...,...,...,...,...
57,2022-03-31,84.439302,4.332782,80.129281
58,2022-06-30,87.082638,5.123172,82.027837
59,2022-09-30,85.022586,4.363500,80.724105
60,2022-12-31,86.553763,5.693156,80.882064


权益基金2-前十大占比

In [6]:
query_sql = """
WITH t AS (
	SELECT
		REPORT_DATE,
		FUND_ID,
		HOLDING_TICKER_SYMBOL,
		HOLDING_SEC_SHORT_NAME,
		RATIO_IN_NA,
        MARKET_VALUE,
		ROW_NUMBER() over ( PARTITION BY REPORT_DATE, FUND_ID, SECURITY_TYPE ORDER BY RATIO_IN_NA DESC ) AS NUMBER_N 
	FROM
		fund_holdings 
	WHERE
		REPORT_DATE >= '20141231' 
		AND SECURITY_TYPE = 'E' 
	ORDER BY
		REPORT_DATE,
		FUND_ID,
		NUMBER_N 
	) SELECT
	b.TICKER_SYMBOL,
	b.SEC_SHORT_NAME,
	t.*,
	d.PREV_TRADE_DATE,
	c.LEVEL_1,
	c.LEVEL_2
FROM
	t
	JOIN fund_info b ON b.FUND_ID = t.FUND_ID and b.IS_MAIN = 1
	JOIN fund_type_own c ON c.TICKER_SYMBOL = b.TICKER_SYMBOL 
	AND c.report_date = t.report_date 
	AND c.LEVEL_1 = '主动权益'
	join md_trade_cal d on d.CALENDAR_DATE = t.REPORT_DATE and d.EXCHANGE_CD = 'XSHG'
WHERE t.NUMBER_N <= 10 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [28]:
df_sum = df.groupby(by=['PREV_TRADE_DATE', 'FUND_ID', 'TICKER_SYMBOL', 'SEC_SHORT_NAME', 'LEVEL_1', 'LEVEL_2']).sum().reset_index()
df_top10_sum_mean = df_sum.groupby(by=['PREV_TRADE_DATE', 'LEVEL_1', 'LEVEL_2'])['RATIO_IN_NA'].mean().reset_index()
df_top10_sum_mean1 = df_top10_sum_mean.pivot_table(
    index='PREV_TRADE_DATE', values="RATIO_IN_NA", columns='LEVEL_1'
)

In [35]:
df_sum

,PREV_TRADE_DATE,FUND_ID,TICKER_SYMBOL,SEC_SHORT_NAME,LEVEL_1,LEVEL_2,RATIO_IN_NA,MARKET_VALUE,NUMBER_N
0,2014-12-30,26,184721,嘉实丰和价值,主动权益,A股主题,27.72,9.907872e+08,55
1,2014-12-30,27,184722,长城久嘉,主动权益,A股主题,23.13,4.704449e+08,55
2,2014-12-30,28,184728,宝盈鸿阳封闭,主动权益,A股主题,44.34,9.270459e+08,55
3,2014-12-30,51,500038,融通通乾封闭,主动权益,A股主题,52.80,1.724796e+09,55
4,2014-12-30,53,500056,易方达科瑞封闭,主动权益,A股主题,28.66,1.037735e+09,55
...,...,...,...,...,...,...,...,...,...
67718,2023-03-30,14935,017445,摩根沪深300指数增强-A,主动权益,指数增强,21.60,3.509901e+06,55
67719,2023-03-30,14937,017648,信澳聚优智选混合-A,主动权益,A股主题,51.52,1.893392e+07,55
67720,2023-03-30,14938,016818,鹏华睿进一年持有期混合-A,主动权益,A股主题,42.43,1.421172e+08,55
67721,2023-03-30,14947,017635,银华动力领航混合-A,主动权益,A股主题,31.98,1.406922e+08,55


In [8]:
df_sum_mean = (
    df.groupby(
        by=['PREV_TRADE_DATE', 'HOLDING_TICKER_SYMBOL', 'HOLDING_SEC_SHORT_NAME']
    ).sum()
    .reset_index()
    .sort_values(by=['PREV_TRADE_DATE', "RATIO_IN_NA"], ascending=False)
)

df_sum_mean['rank'] = df_sum_mean.groupby(
        by=['PREV_TRADE_DATE']
    )["MARKET_VALUE"].rank(ascending=False)

df_sum = df_sum_mean.groupby(
        by=['PREV_TRADE_DATE']
    )["MARKET_VALUE"].sum().reset_index().rename(columns={"MARKET_VALUE": "MARKET_VALUE_SUM"})
df_sum_mean=  df_sum_mean.merge(df_sum)
df_sum_mean["比例"] = (df_sum_mean['MARKET_VALUE']/df_sum_mean['MARKET_VALUE_SUM'])*100

In [39]:
df_sum_mean

,PREV_TRADE_DATE,HOLDING_TICKER_SYMBOL,HOLDING_SEC_SHORT_NAME,FUND_ID,RATIO_IN_NA,MARKET_VALUE,NUMBER_N,rank,MARKET_VALUE_SUM,比例,PREV_TRADE_DATE1
0,2023-03-30,600519,贵州茅台,7479669,6573.68,1.043465e+11,3041,1.0,2.348359e+12,4.443379,2023-03
1,2023-03-30,300750,宁德时代,6365696,4641.28,7.420996e+10,3134,2.0,2.348359e+12,3.160077,2023-03
2,2023-03-30,00700,腾讯控股,5450410,3379.58,5.439527e+10,1782,5.0,2.348359e+12,2.316310,2023-03
3,2023-03-30,000568,泸州老窖,4153984,3362.59,5.700851e+10,2516,3.0,2.348359e+12,2.427589,2023-03
4,2023-03-30,000858,五粮液,4258315,3293.20,5.487804e+10,2619,4.0,2.348359e+12,2.336867,2023-03
...,...,...,...,...,...,...,...,...,...,...,...
51824,2014-12-30,601519,大智慧,716,0.76,3.119520e+05,2,992.0,5.165887e+11,0.000060,2014-12
51825,2014-12-30,601216,君正集团,716,0.65,2.651760e+05,4,993.0,5.165887e+11,0.000051,2014-12
51826,2014-12-30,600816,ST安信,1215,0.46,2.030746e+05,7,994.0,5.165887e+11,0.000039,2014-12
51827,2014-12-30,600551,时代出版,1215,0.45,1.999666e+05,9,995.0,5.165887e+11,0.000039,2014-12


权益基金-公募基金前50/100大占比

In [9]:
df_sum_mean.query("rank <= 100").groupby("PREV_TRADE_DATE")['比例'].sum()

PREV_TRADE_DATE
2014-12-30    61.498253
2015-03-30    53.545512
2015-06-29    44.778011
2015-09-29    48.374712
2015-12-30    40.671102
2016-03-30    40.865623
2016-06-29    39.402863
2016-09-29    40.493253
2016-12-30    38.118547
2017-03-30    43.440195
2017-06-29    53.067164
2017-09-29    58.310889
2017-12-29    65.853673
2018-03-30    64.319405
2018-06-29    63.930945
2018-09-28    64.825003
2018-12-28    61.654282
2019-03-29    64.659271
2019-06-28    71.013273
2019-09-27    70.353853
2019-12-30    69.044482
2020-03-30    67.025863
2020-06-29    67.838059
2020-09-29    69.852704
2020-12-30    74.874446
2021-03-30    72.438966
2021-06-29    71.593823
2021-09-29    65.091279
2021-12-30    63.202569
2022-03-30    61.606522
2022-06-29    63.034054
2022-09-29    59.667935
2022-12-30    57.592471
2023-03-30    54.211082
Name: 比例, dtype: float64

权益基金-前十大风格

In [10]:
barra_exposure_sql = """
SELECT
	a.*
FROM
	dy1d_exposure_cne6 a
	join (SELECT DISTINCT
		quarter_end_date 
	FROM
		md_trade_cal 
WHERE
	EXCHANGE_CD = 'XSHG' 
	AND quarter_end_date BETWEEN '20141230' 
AND '20230331') t on a.TRADE_DATE = t.quarter_end_date
"""
barra_exposure = DB_CONN_DATAYES.exec_query(barra_exposure_sql).drop(columns=['ID'])
barra_exposure["TRADE_DATE"] = barra_exposure["TRADE_DATE"].apply(lambda s: parse(s).strftime("%Y-%m"))

In [11]:
barra_exposure = barra_exposure.dropna(axis=1)

In [12]:
barra_exposure.loc[:, "BETA": "COUNTRY"] = barra_exposure.loc[:, "BETA": "COUNTRY"].astype("float")

In [13]:
df_sum_mean["PREV_TRADE_DATE1"] = df_sum_mean["PREV_TRADE_DATE"].apply(lambda s: s.strftime("%Y-%m"))

In [14]:
(
    df_sum_mean.merge(
        barra_exposure, 
        left_on=['HOLDING_TICKER_SYMBOL', "PREV_TRADE_DATE1"], right_on=['TICKER_SYMBOL', "TRADE_DATE"]
    ).groupby(by='PREV_TRADE_DATE1')
    .mean()
    .rename(columns=BARRA_SW21_FACTOR_NAME_DICT)
    .to_excel("d:/barra.xlsx")
)

In [15]:
df_barra = df_sum_mean.merge(
        barra_exposure, 
        left_on=['HOLDING_TICKER_SYMBOL', "PREV_TRADE_DATE1"], right_on=['TICKER_SYMBOL', "TRADE_DATE"]
    )

In [16]:
col = df_barra.columns

In [17]:
for column in col[13:-2]:
    df_barra.loc[:, column] = df_barra.loc[:, column]*df_barra.loc[:, "比例"]

In [18]:
df1 = (df_barra.groupby(by='PREV_TRADE_DATE1')
    .sum()

)

In [19]:
for column in col[13:-2]:
    df1.loc[:, column] = df1.loc[:, column]/df1.loc[:, "比例"]

In [20]:
df1.rename(columns=BARRA_SW21_FACTOR_NAME_DICT).to_excel("D:/barra.xlsx")

In [21]:
col[13]

'BETA'